In [6]:
import torch
from torch import nn
from d2l import torch as d2l

x=torch.normal(1,2,(3,4))
x
x.shape[0],x.shape[1],x


(3,
 4,
 tensor([[ 1.3210,  1.1308, -1.2839,  2.3363],
         [ 0.6996, -0.0933, -1.8603, -1.6526],
         [ 3.9263,  2.1633,  1.3732,  3.3151]]))

In [9]:
def corr2d(x,k):#定义卷积
    h,w=k.shape
    y=torch.zeros(x.shape[0]-h+1,x.shape[1]-w+1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i][j]=(x[i:i+h,j:j+w]*k).sum()
    return y

In [10]:
x=torch.tensor([[0.,1.,2.],[3.,4.,5.],[6.,7.,8.]])
k=torch.tensor([[0.,1.],[2.,3.]])
corr2d(x,k)

tensor([[19., 25.],
        [37., 43.]])

In [11]:
#定义卷积层：
class Conv2d(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))#用广播机制
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [14]:
x=torch.ones((6,8))
x[:,2:6]=0
x

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [18]:
k=torch.tensor([[1.,-1.]])

In [24]:
y=corr2d(x,k)
y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [21]:
corr2d(x.T,k)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### 如上，k卷积核只能检测垂直边缘，不能检测水平边缘

In [33]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
x=x.reshape((1,1,6,8))
y=y.reshape((1,1,6,7))
lr=3e-2
for i in range(10):
    y_hat=conv2d(x)
    loss=(y_hat-y.reshape(y_hat.shape))**2
    conv2d.zero_grad()
    loss.sum().backward()
    conv2d.weight.data-=lr*conv2d.weight.grad
    print(f'epoch{i+1},loss{loss.sum():.3f}')

epoch1,loss13.221
epoch2,loss5.432
epoch3,loss2.236
epoch4,loss0.923
epoch5,loss0.382
epoch6,loss0.159
epoch7,loss0.067
epoch8,loss0.029
epoch9,loss0.012
epoch10,loss0.006


In [34]:
conv2d.state_dict()

OrderedDict([('weight', tensor([[[[ 0.9842, -0.9917]]]]))])